# Lecture Rio Primero

In [ ]:
import os
import numpy as np
import flopy.modflow as mf

In [ ]:
# Set name for MODFLOW model
modelname = 'RioPrimero'

In [ ]:
model = mf.Modflow(modelname='RP1', exe_name='mf2005', version='mf2005', model_ws='data')

In [ ]:
# Model domain and grid definition
Lx = 11250.0  #aquifer length in x-direction
Ly = 6000.0  #aquifer length in y-direction
nlay = 3  #number of layers
nrow = 6  #number of rows
ncol = 10  #number of columns

delr = Lx / ncol  #cell width along x-direction
delc = Ly / nrow  #cell width along y-direction

dis = mf.ModflowDis(model, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=460,
                    botm=[450, 448, 390], nper=1, perlen=1827, nstp=1, steady=True, itmuni=4, lenuni=2)
dis

In [ ]:
dis.plot()

In [ ]:
bas = mf.ModflowBas(model, ibound=1, strt=460)
bas.plot()

In [ ]:
ghb_west_sp_data = {0: [
  [0, 0, 0, 450, 200],
  [0, 1, 0, 450, 200],
  [0, 2, 0, 450, 200],
  [0, 3, 0, 450, 200],
  [0, 4, 0, 450, 200],
  [0, 5, 0, 450, 200],
  [0, 0, 9, 445, 200],
  [0, 1, 9, 445, 200],
  [0, 2, 9, 445, 200],
  [0, 3, 9, 445, 200],
  [0, 4, 9, 445, 200],
  [0, 5, 9, 445, 200],
]}

ghb = mf.ModflowGhb(model, stress_period_data=ghb_west_sp_data)
ghb.plot()

In [ ]:
rch = mf.ModflowRch(model, rech=0.00033)

In [ ]:
riv_sp_data = {0: [
  [0, 5, 1, 455, 86.4, 453],
  [0, 4, 2, 455, 86.4, 453],
  [0, 4, 3, 455, 86.4, 453],
  [0, 3, 4, 455, 86.4, 453],
  [0, 2, 5, 455, 86.4, 453],
  [0, 3, 6, 455, 86.4, 453],
  [0, 3, 7, 455, 86.4, 453],
  [0, 2, 8, 455, 86.4, 453],
  [0, 1, 9, 455, 86.4, 453],
]}
riv = mf.ModflowRiv(model, stress_period_data=riv_sp_data)

In [ ]:
# Well data
wel = mf.ModflowWel(model, stress_period_data={
  0: [
    [0, 1, 1, -5000],
    [0, 0, 2, -10000],
    [0, 5, 5, -5000],
    [0, 0, 8, -5000],
    [0, 1, 6, -16000],
  ]
})


In [ ]:
# Flow data
lpf = mf.ModflowLpf(model, hk=[8.64, 1, 8.64], vka=[0.864, 0.1, 0.864], ss=1e-5, sy=0.2)

In [ ]:
pcg = mf.ModflowPcg(model, rclose=1e-2, hclose=1e-2)
oc = mf.ModflowOc(model)

In [ ]:
# Write the MODFLOW model input files

In [ ]:
model.write_input()
model.run_model()

In [ ]:
# Post process the results

In [ ]:
import flopy.utils as fu

hds = fu.HeadFile(os.path.join('data', 'RP1.hds'))
h = hds.get_data(kstpkper=(0, 0))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

x = np.linspace(0, delr * ncol, ncol)
y = np.linspace(delc * nrow, 0, nrow)

c = plt.contour(x, y, h[0], np.arange(h[0].min(), h[0].max(), 1))
plt.clabel(c, fmt='%2.1f')
plt.axis('scaled')
plt.plot()

In [ ]:
plt.imshow(h[0])